In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

#import gensim
#from gensim.summarization import summarize
#for importing and manuplating data
import numpy as np 
import pandas as pd 
import string
import csv
from googlesearch import search
#for fitting model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
#connecting with google, request data and apllying nlp
import requests
from lxml import html
from bs4 import BeautifulSoup
import string
import urllib.request 
from urllib.request import urlopen
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


# Create your views here.


def tt(input_query):
  USE_CUDA = torch.cuda.is_available()
  device = torch.device("cuda" if USE_CUDA else "cpu")
  def listToString(s):
    str1 = ""  
    for ele in s: 
        str1 += ele + " "
    return str1 
  def cleaner(x):
    return [a for a in (''.join([a for a in x if a not in string.punctuation])).lower().split()]
  
  csv_file = "Classification_dataset_cureya.csv"
  df = pd.read_csv(csv_file)
  Pipe = Pipeline([
      ('bow',CountVectorizer(analyzer=cleaner)),
      ('tfidf',TfidfTransformer()),
      ('classifier',DecisionTreeClassifier())
  ])
  Pipe.fit(df['patterns'],df['tag'])

  def spacy_process_i(text):
    doc = nlp(text)
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    filtered_sentence =[] 
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)
    
    counter_string = listToString(filtered_sentence)
    pp_string = counter_string.replace("-PRON-", "")
    
    return pp_string
  
  def loadLines(fileName, fields):
      lines = {}
      with open(fileName, 'r', encoding='iso-8859-1') as f:
          for line in f:
              values = line.split(" +++$+++ ")
              lineObj = {}
              for i, field in enumerate(fields):
                  lineObj[field] = values[i]
              lines[lineObj['lineID']] = lineObj
      return lines


  def loadConversations(fileName, lines, fields):
      conversations = []
      with open(fileName, 'r', encoding='iso-8859-1') as f:
          for line in f:
              values = line.split(" +++$+++ ")
              convObj = {}
              for i, field in enumerate(fields):
                  convObj[field] = values[i]

              utterance_id_pattern = re.compile('L[0-9]+')
              lineIds = utterance_id_pattern.findall(convObj["utteranceIDs"])
              convObj["lines"] = []
              for lineId in lineIds:
                  convObj["lines"].append(lines[lineId])
              conversations.append(convObj)
      return conversations


  def extractSentencePairs(conversations):
      qa_pairs = []
      for conversation in conversations:
          for i in range(len(conversation["lines"]) - 1): 
              inputLine = conversation["lines"][i]["text"].strip()
              targetLine = conversation["lines"][i+1]["text"].strip()
              if inputLine and targetLine:
                  qa_pairs.append([inputLine, targetLine])
      return qa_pairs


  PAD_token = 0  
  SOS_token = 1  
  EOS_token = 2 

  class Voc:
      def __init__(self, name):
          self.name = name
          self.trimmed = False
          self.word2index = {}
          self.word2count = {}
          self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
          self.num_words = 3  

      def addSentence(self, sentence):
          for word in sentence.split(' '):
              self.addWord(word)

      def addWord(self, word):
          if word not in self.word2index:
              self.word2index[word] = self.num_words
              self.word2count[word] = 1
              self.index2word[self.num_words] = word
              self.num_words += 1
          else:
              self.word2count[word] += 1

      def trim(self, min_count):
          if self.trimmed:
              return
          self.trimmed = True

          keep_words = []

          for k, v in self.word2count.items():
              if v >= min_count:
                  keep_words.append(k)

          print('keep_words {} / {} = {:.4f}'.format(
              len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
          ))

          
          self.word2index = {}
          self.word2count = {}
          self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
          self.num_words = 3 

          for word in keep_words:
              self.addWord(word)


  MAX_LENGTH = 10
  def unicodeToAscii(s):
      return ''.join(
          c for c in unicodedata.normalize('NFD', s)
          if unicodedata.category(c) != 'Mn'
      )

  def normalizeString(s):
      s = unicodeToAscii(s.lower().strip())
      s = re.sub(r"([.!?])", r" \1", s)
      s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
      s = re.sub(r"\s+", r" ", s).strip()
      return s

  def readVocs(datafile, corpus_name):
      print("Reading lines...")
      lines = open(datafile, encoding='utf-8').\
          read().strip().split('\n')
      pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
      voc = Voc(corpus_name)
      return voc, pairs

  def filterPair(p):
      return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH


  def filterPairs(pairs):
      return [pair for pair in pairs if filterPair(pair)]

  def loadPrepareData(corpus, corpus_name, datafile, save_dir):
      print("Start preparing training data ...")
      voc, pairs = readVocs(datafile, corpus_name)
      print("Read {!s} sentence pairs".format(len(pairs)))
      pairs = filterPairs(pairs)
      print("Trimmed to {!s} sentence pairs".format(len(pairs)))
      print("Counting words...")
      for pair in pairs:
          voc.addSentence(pair[0])
          voc.addSentence(pair[1])
      print("Counted words:", voc.num_words)
      return voc, pairs


  MIN_COUNT = 3   

  def trimRareWords(voc, pairs, MIN_COUNT):
      voc.trim(MIN_COUNT)
      keep_pairs = []
      for pair in pairs:
          input_sentence = pair[0]
          output_sentence = pair[1]
          keep_input = True
          keep_output = True
          for word in input_sentence.split(' '):
              if word not in voc.word2index:
                  keep_input = False
                  break
          for word in output_sentence.split(' '):
              if word not in voc.word2index:
                  keep_output = False
                  break

          if keep_input and keep_output:
              keep_pairs.append(pair)

      print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
      return keep_pairs

  def indexesFromSentence(voc, sentence):
      return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


  def zeroPadding(l, fillvalue=PAD_token):
      return list(itertools.zip_longest(*l, fillvalue=fillvalue))

  def binaryMatrix(l, value=PAD_token):
      m = []
      for i, seq in enumerate(l):
          m.append([])
          for token in seq:
              if token == PAD_token:
                  m[i].append(0)
              else:
                  m[i].append(1)
      return m

  def inputVar(l, voc):
      indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
      lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
      padList = zeroPadding(indexes_batch)
      padVar = torch.LongTensor(padList)
      return padVar, lengths

  def outputVar(l, voc):
      indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
      max_target_len = max([len(indexes) for indexes in indexes_batch])
      padList = zeroPadding(indexes_batch)
      mask = binaryMatrix(padList)
      mask = torch.BoolTensor(mask)
      padVar = torch.LongTensor(padList)
      return padVar, mask, max_target_len

  def batch2TrainData(voc, pair_batch):
      pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
      input_batch, output_batch = [], []
      for pair in pair_batch:
          input_batch.append(pair[0])
          output_batch.append(pair[1])
      inp, lengths = inputVar(input_batch, voc)
      output, mask, max_target_len = outputVar(output_batch, voc)
      return inp, lengths, output, mask, max_target_len


  class EncoderRNN(nn.Module):
      def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
          super(EncoderRNN, self).__init__()
          self.n_layers = n_layers
          self.hidden_size = hidden_size
          self.embedding = embedding
          self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                            dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

      def forward(self, input_seq, input_lengths, hidden=None):
          embedded = self.embedding(input_seq)
          packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
          outputs, hidden = self.gru(packed, hidden)
          outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
          
          outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
      
          return outputs, hidden


  class Attn(nn.Module):
      def __init__(self, method, hidden_size):
          super(Attn, self).__init__()
          self.method = method
          if self.method not in ['dot', 'general', 'concat']:
              raise ValueError(self.method, "is not an appropriate attention method.")
          self.hidden_size = hidden_size
          if self.method == 'general':
              self.attn = nn.Linear(self.hidden_size, hidden_size)
          elif self.method == 'concat':
              self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
              self.v = nn.Parameter(torch.FloatTensor(hidden_size))

      def dot_score(self, hidden, encoder_output):
          return torch.sum(hidden * encoder_output, dim=2)

      def general_score(self, hidden, encoder_output):
          energy = self.attn(encoder_output)
          return torch.sum(hidden * energy, dim=2)

      def concat_score(self, hidden, encoder_output):
          energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
          return torch.sum(self.v * energy, dim=2)

      def forward(self, hidden, encoder_outputs):
          if self.method == 'general':
              attn_energies = self.general_score(hidden, encoder_outputs)
          elif self.method == 'concat':
              attn_energies = self.concat_score(hidden, encoder_outputs)
          elif self.method == 'dot':
              attn_energies = self.dot_score(hidden, encoder_outputs)

          attn_energies = attn_energies.t()


          return F.softmax(attn_energies, dim=1).unsqueeze(1)



  class LuongAttnDecoderRNN(nn.Module):
      def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
          super(LuongAttnDecoderRNN, self).__init__()

          self.attn_model = attn_model
          self.hidden_size = hidden_size
          self.output_size = output_size
          self.n_layers = n_layers
          self.dropout = dropout

          self.embedding = embedding
          self.embedding_dropout = nn.Dropout(dropout)
          self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
          self.concat = nn.Linear(hidden_size * 2, hidden_size)
          self.out = nn.Linear(hidden_size, output_size)

          self.attn = Attn(attn_model, hidden_size)

      def forward(self, input_step, last_hidden, encoder_outputs):
          embedded = self.embedding(input_step)
          embedded = self.embedding_dropout(embedded)
          rnn_output, hidden = self.gru(embedded, last_hidden)
          attn_weights = self.attn(rnn_output, encoder_outputs)
          context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
          rnn_output = rnn_output.squeeze(0)
          context = context.squeeze(1)
          concat_input = torch.cat((rnn_output, context), 1)
          concat_output = torch.tanh(self.concat(concat_input))
          output = self.out(concat_output)
          output = F.softmax(output, dim=1)
          return output, hidden


  def maskNLLLoss(inp, target, mask):
      nTotal = mask.sum()
      crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
      loss = crossEntropy.masked_select(mask).mean()
      loss = loss.to(device)
      return loss, nTotal.item()


  def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
            encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

      encoder_optimizer.zero_grad()
      decoder_optimizer.zero_grad()

      input_variable = input_variable.to(device)
      target_variable = target_variable.to(device)
      mask = mask.to(device)
      lengths = lengths.to("cpu")

      loss = 0
      print_losses = []
      n_totals = 0

      encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

      decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
      decoder_input = decoder_input.to(device)

      decoder_hidden = encoder_hidden[:decoder.n_layers]

      use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

      if use_teacher_forcing:
          for t in range(max_target_len):
              decoder_output, decoder_hidden = decoder(
                  decoder_input, decoder_hidden, encoder_outputs
              )
              decoder_input = target_variable[t].view(1, -1)
              mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
              loss += mask_loss
              print_losses.append(mask_loss.item() * nTotal)
              n_totals += nTotal
      else:
          for t in range(max_target_len):
              decoder_output, decoder_hidden = decoder(
                  decoder_input, decoder_hidden, encoder_outputs
              )
              
              _, topi = decoder_output.topk(1)
              decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
              decoder_input = decoder_input.to(device)
              
              mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
              loss += mask_loss
              print_losses.append(mask_loss.item() * nTotal)
              n_totals += nTotal


      loss.backward()

      _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
      _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

      encoder_optimizer.step()
      decoder_optimizer.step()

      return sum(print_losses) / n_totals



  def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

      training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                        for _ in range(n_iteration)]

      print('Initializing ...')
      start_iteration = 1
      print_loss = 0
      if loadFilename:
          start_iteration = checkpoint['iteration'] + 1

      print("Training...")
      for iteration in range(start_iteration, n_iteration + 1):
          training_batch = training_batches[iteration - 1]
          input_variable, lengths, target_variable, mask, max_target_len = training_batch

          loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                      decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
          print_loss += loss

          if iteration % print_every == 0:
              print_loss_avg = print_loss / print_every
              print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
              print_loss = 0

          if (iteration % save_every == 0):
              directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
              if not os.path.exists(directory):
                  os.makedirs(directory)
              torch.save({
                  'iteration': iteration,
                  'en': encoder.state_dict(),
                  'de': decoder.state_dict(),
                  'en_opt': encoder_optimizer.state_dict(),
                  'de_opt': decoder_optimizer.state_dict(),
                  'loss': loss,
                  'voc_dict': voc.__dict__,
                  'embedding': embedding.state_dict()
              }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))


  class GreedySearchDecoder(nn.Module):
      def __init__(self, encoder, decoder):
          super(GreedySearchDecoder, self).__init__()
          self.encoder = encoder
          self.decoder = decoder

      def forward(self, input_seq, input_length, max_length):
          encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
          decoder_hidden = encoder_hidden[:decoder.n_layers]
          decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
          all_tokens = torch.zeros([0], device=device, dtype=torch.long)
          all_scores = torch.zeros([0], device=device)
          for _ in range(max_length):
              decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
              decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
              all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
              all_scores = torch.cat((all_scores, decoder_scores), dim=0)
              decoder_input = torch.unsqueeze(decoder_input, 0)
          return all_tokens, all_scores


  def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
      indexes_batch = [indexesFromSentence(voc, sentence)]
      lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
      input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
      input_batch = input_batch.to(device)
      lengths = lengths.to("cpu")
      tokens, scores = searcher(input_batch, lengths, max_length)
      decoded_words = [voc.index2word[token.item()] for token in tokens]
      return decoded_words

  def evaluateInput(input_s,encoder, decoder, searcher, voc):
    input_sentence = ''
    try:
      input_sentence = input_s
      if input_sentence == 'q' or input_sentence == 'quit':
        return("End")
      input_sentence = normalizeString(input_sentence)
      output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
      output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
      return listToString(output_words)

    except KeyError:
      return("Error: Encountered unknown word.")


  corpus_name = "cornell movie-dialogs corpus"
  corpus = "p/data"

  def printLines(file, n=10):
      with open(file, 'rb') as datafile:
          lines = datafile.readlines()
      for line in lines[:n]:
          print(line)




  datafile = os.path.join(corpus, "formatted_movie_lines.txt")
  save_dir = os.path.join("data", "save")
  voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)  
  model_name = 'cb_model'
  attn_model = 'dot'
  hidden_size = 500
  encoder_n_layers = 2
  decoder_n_layers = 2
  dropout = 0.1
  batch_size = 64

  loadFilename = "4000_checkpoint.tar"
  checkpoint_iter = 4000


  if loadFilename:
      checkpoint = torch.load(loadFilename)
      encoder_sd = checkpoint['en']
      decoder_sd = checkpoint['de']
      encoder_optimizer_sd = checkpoint['en_opt']
      decoder_optimizer_sd = checkpoint['de_opt']
      embedding_sd = checkpoint['embedding']
      voc.__dict__ = checkpoint['voc_dict']


  print('Building encoder and decoder ...')
  embedding = nn.Embedding(voc.num_words, hidden_size)
  if loadFilename:
      embedding.load_state_dict(embedding_sd)
  encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
  decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
  if loadFilename:
      encoder.load_state_dict(encoder_sd)
      decoder.load_state_dict(decoder_sd)
  encoder = encoder.to(device)
  decoder = decoder.to(device)
  print('Models built and ready to go!')

  encoder.eval()
  decoder.eval()

  searcher = GreedySearchDecoder(encoder, decoder)

  def search_web(query):
    search_result_list1 = []
    search_result_list2 = []
    search_result_list3 = []
    disease = query
    website = ['mayo clinic', 'apollo clinic','cdc']
    query1 =disease + " " + website[0] 
    #print(query1)
    search_result_list1.append(list(search(query1, tld="co.in", num=3, stop=5, pause=2)))
    #print(search_result_list1)
    query2 = disease + " " + website[1] 
    #print(query2)
    search_result_list2.append(list(search(query2, tld="co.in", num=3, stop=5, pause=2)))
    #print(search_result_list2)
    query3 = disease + " " + website[2] 
    #print(query3)
    search_result_list3.append(list(search(query3, tld="co.in", num=3, stop=5, pause=2)))
    #print(search_result_list3)

    return search_result_list1,search_result_list2,search_result_list3

  def mayo_info(html):
    soup = BeautifulSoup(html, 'html.parser')
    list1 = []
    for header in soup.find_all('p'):
        nextNode = header
        if(True):
          #print("heaad"+header.text.strip())
          list1.append(header.text.strip())
          while True:
              nextNode = nextNode.nextSibling
              if nextNode is None:
                  break
              if isinstance(nextNode, NavigableString):
                  #list1.append(nextNode.strip())
                  print("")
                  #print("bye")
                  #print(nextNode.strip())
              if isinstance(nextNode, Tag):
                  if nextNode.name == "p":
                      break
                  #print (nextNode.get_text(strip=True).strip())
                  #print(list1)
                  #print("hei")
                  #list1.append(nextNode.get_text(strip=True).strip())

    #print(list1)
    reference = listToString(list1)
    reference = reference.replace(' and the triple-shield Mayo Clinic logo are trademarks of Mayo Foundation for Medical Education and Research.', '')
    reference = reference.replace('Check out these best-sellers and special offers on books and newsletters from Mayo Clinic Press.', '')
    reference = reference.replace('Mayo Clinic does not endorse companies or products. Advertising revenue supports our not-for-profit mission.', '')
    reference = reference.replace('We are open for safe in-person care.', '')
    reference = reference.replace('Featured conditions', '')
    reference = reference.replace('Any use of this site constitutes your agreement to the Terms and Conditions and Privacy Policy linked below.\n\nTerms and Conditions\nPrivacy Policy\nNotice of Privacy Practices\nNotice of Nondiscrimination\nManage Cookies', '')
    reference = reference.replace('MayoClinic.org', '')
    reference = reference.replace('Mayo Foundation ', '')
    reference = reference.replace('This site complies with the  HONcode standard for trustworthy health information: verify here.', '')
    reference = reference.lower()
    reference = reference.replace('mayo', '')
    reference = reference.replace('clinic', '')
    reference = reference.replace('","', '')
    reference = reference.replace('a single copy of these materials may be reprinted for noncommercial personal use only.', '')
    reference = reference.replace('  ', '')
    reference = reference.replace('"', '')
    #print(reference)
    flag = 0
    sol = []
    # traverse paragraphs from soup
    for data in soup.find_all("h2"):
        if(data.text.strip() == 'Prevention' or data.text.strip() == 'Overview' or data.text.strip() == 'Symptoms' or data.text.strip() == 'Causes' or data.text.strip() == 'Complications' or data.text.strip() == 'Risk factors' or data.text.strip() == 'Diagnosis' or data.text.strip() == 'Treatment'):
          sol.append(data.text.strip())
          sol.append("\n")
          #print(data.text.strip())
          para = data.find_next_sibling('ul')
          sol.append(para.text)
          #print(para.text)
          flag = 1

    if flag == 0:
      resultant = soup.findAll('h2')
      for i in range(len(resultant)):
        sol.append(resultant[i].text)

      #print(sol)

    sol = list(dict.fromkeys(sol))
    reference_counter = listToString(sol)

    #print("\nReference: ", *search_result_list)
    return reference,reference_counter


  def cdc_info(html):
    soup = BeautifulSoup(html, 'html.parser')
    list = []
    for header in soup.find_all('p'):
        nextNode = header
        if(True):
          #print("heaad"+header.text.strip())
          list.append(header.text.strip())
          while True:
              nextNode = nextNode.nextSibling
              if nextNode is None:
                  break
              if isinstance(nextNode, NavigableString):
                print("")
                #list.append(nextNode.strip())
              if isinstance(nextNode, Tag):
                  if nextNode.name == "p":
                      break
                  #print (nextNode.get_text(strip=True).strip())
                  #print(list)
                  #print("hi")
                  #list.append(nextNode.get_text(strip=True).strip())

    reference = listToString(list)
    reference = reference.replace('Your email address will not be published. Required fields are marked', '')
    reference = reference.replace('Comment', '')
    reference = reference.replace('Name', '')
    reference = reference.replace('Website', '')
    reference = reference.replace('*', '')
    reference = reference.lower()
    reference = reference.replace('all rights reserved', '')
    #print(reference)
    return reference


  def apollo_info(html):
    soup = BeautifulSoup(html, 'html.parser')
    list = []
    for header in soup.find_all('p'):
        nextNode = header
        if(True):
          #print("heaad"+header.text.strip())
          list.append(header.text.strip())
          while True:
              nextNode = nextNode.nextSibling
              if nextNode is None:
                  break
              if isinstance(nextNode, NavigableString):
                print("")
                #list.append(nextNode.strip())
              if isinstance(nextNode, Tag):
                  if nextNode.name == "p":
                      break
                  #print (nextNode.get_text(strip=True).strip())
                  #print(list)
                  #print("hi")
                  #list.append(nextNode.get_text(strip=True).strip())

    reference = listToString(list)
    reference = reference.replace('Your email address will not be published. Required fields are marked', '')
    reference = reference.replace('Comment', '')
    reference = reference.replace('Name', '')
    reference = reference.replace('Website', '')
    reference = reference.replace('*', '')
    reference = reference.lower()
    reference = reference.replace('all rights reserved', '')
    reference = reference.replace('apollo hospitals enterprise limited', '')
    reference = reference.replace('apollo hospitals', '')
    reference = reference.replace('https://www.apolloclinic.com/clinic-locator', '')
    reference = reference.replace('email', '')
    #print(reference)
    return reference

  def get_context(search_result_list1,search_result_list2,search_result_list3):
    url = search_result_list1[0]
    r = requests.get(url[0])
    html = r.text
    context_mayo,context_mayo2 = mayo_info(html)
    url2 = search_result_list2[0]
    r2 = requests.get(url2[0])
    html2 = r2.text
    context_apollo = apollo_info(html2)
    url3 = search_result_list3[0]
    r3 = requests.get(url3[0])
    html3 = r3.text
    context_cdc = cdc_info(html3)
    return context_apollo,context_mayo,context_mayo2,context_cdc

  nlp = spacy.load("en_core_web_sm")

  def spacy_process(text):
    doc = nlp(text)
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    #print("Tokenize+Lemmatize:")
    #print(lemma_list)
    
    #removes stop-word like at , i , am ,etc

    #Filter the stopword
    #filtered_sentence =[] 
    #for word in lemma_list:
        #lexeme = nlp.vocab[word]
        #if lexeme.is_stop == False:
            #filtered_sentence.append(word) 
    
    #Remove punctuation
    #punctuations="?:!.,;"
    #for word in filtered_sentence:
        #if word in punctuations:
            #filtered_sentence.remove(word)
    
    counter_string = listToString(lemma_list)
    pp_string = counter_string.replace("-PRON-", "")
    
    return pp_string


  def qadiseasebert(question,text):
    tokenizer = AutoTokenizer.from_pretrained("biobert_v1.1_pubmed_squad_v2",model_max_length=512)
    model = AutoModelForQuestionAnswering.from_pretrained("biobert_v1.1_pubmed_squad_v2")
    inputs = tokenizer(question, text, add_special_tokens=True, truncation=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    #print(f"Question: {question}")
    answer = answer.replace("[CLS]","")
    answer = answer.replace("[SEP]","")
    #print(answer)
    answer = answer.replace(question,"")
    answer = answer.replace(spacy_process(question),"")
    #print(answer)
    #print(f"Answer: {answer}")
    return answer

  def txt_pp(question,text):
    counter = spacy_process(question)
    res = text.split()
    #print(len(res))
    '''
    if(len(res)>400):
      text = spacy_process(text)
      res = text.split()
      #print(len(res))
      if(len(res)>300):
        text=summarize(text,word_count=400)
    text = counter+" "+text
    '''
    return question,text

  def qadisease_c(query):
    list1,list2,list3 = search_web(query)
    context1,context2,context2_1,context3 = get_context(list1,list2,list3)
    _,ctxt1=txt_pp(query,context1)
    _,ctxt2=txt_pp(query,context2)
    _,ctxt3=txt_pp(query,context2_1)
    _,ctxt4=txt_pp(query,context3)
    ans1 = qadiseasebert(query,ctxt1)
    ans2 = qadiseasebert(query,ctxt2)
    ans3 = qadiseasebert(query,ctxt3)
    ans4 = qadiseasebert(query,ctxt4)
    score = []
    ans_t = []
    counter_q = nlp(spacy_process(query))
    score.append(counter_q.similarity(nlp(ans1)))
    score.append(counter_q.similarity(nlp(ans2)))
    score.append(counter_q.similarity(nlp(ans3)))
    score.append(counter_q.similarity(nlp(ans4)))
    ans_t.append(ans1)
    ans_t.append(ans2)
    ans_t.append(ans3)
    ans_t.append(ans4)
    print(score)
    return score,ans_t

  def f_ans(score,ans):
    fallback = "Couldnt understand your query, Please try again"
    maxpos = score.index(max(score))
    if maxpos == 0:
      return ans[0]
    elif maxpos == 1:
      return ans[1]
    elif maxpos == 2:
      return ans[2]
    elif maxpos == 3:
      return ans[3]
    else:
      return fallback

  def reemovNestings(l): 
      for i in l:
          if type(i) == list:
              reemovNestings(i)
          else:
              output.append(i)

  def servicecard(lst):
    lst_p = []
    def reemovNestings(l): 
      for i in l:
          if type(i) == list:
              reemovNestings(i)
          else:
              lst_p.append(i)
    reemovNestings(lst)
    keywords_check = ["yoga","excercise","diet","nutrition","nutritional","nutritionist","dietician","consultation","treatment","alternative treatment","weight management","muscle gain","pcod","cardiovascular disease","renal","anaemia","gastrointestinal","disease","ayurveda","ayurvedic","naturopathy","homeopathy","unani","siddha","cureya","doctor","blood pressure","heart","artery","overweight","underweight","pcos","thyroid","cardiovascular","dialysis","typhoid","influenza","malaria","aids"]
    set_a = set(lst_p)
    set_b = set(keywords_check)
    set_c = set_a & set_b
    output = list(set_c)
    output = listToString(output)
    return output

  counter = 1
  msg_type = 0
  msg_track = []
  url1 = ""
  url2 = ""
  price = ""
  while(counter == 1):
    query = input_query
    output = query
    if(output == "q" or output == "quit" or output == "end" or output == "exit"):
      counter = 0
      print("Thanks")
      break

    output = Pipe.predict([spacy_process_i(query)])[0]
    #print(output)
    msg_track.append(spacy_process_i(query).split(" "))
    if(len(msg_track)<3):
      if(output == "medical"):
        msg_type = 1
        op,at = qadisease_c(query)
        answer = f_ans(op,at)
        print(answer)
        #print(msg_type)
        return msg_type,answer,url1,url2,price 
      elif(output == "general"):
        msg_type = 1
        answer = evaluateInput(query,encoder, decoder, searcher, voc)
        print(answer)
        #print(msg_type)
        return msg_type,answer,url1,url2,price 
      elif(output == "service"):
        msg_type = 2
        answer = "service"
        url1 = "req1"
        url2 = "req2"
        price = 450
        print(answer)
        return msg_type,answer,url1,url2,price 
    else:
      msg_type = 2
      answer = servicecard(msg_track)
      #print(msg_type)
      print("\nCARD-services\n")
      print(answer)
      answer = "service"
      url1 = "req1"
      url2 = "req2"
      price = 450
      return msg_type,answer,url1,url2,price 

  return msg_type,answer,url1,url2,price 

In [2]:
q,p,r,s,t = tt("prevention of diabetes")

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008
Building encoder and decoder ...
Models built and ready to go!































































































































[0.45170933525764906, 0.0, 0.0, 0.0]
   pre - diabetes, as the name suggests occurs before the diabetes phase and has the potential to evolve into a fully fledged type 2 diabetes down the road. the most frightening aspect of pre - diabetes is that it has no symptoms unless a manual blood test is done. during this condition, the body functions normally, just that the glucose level in the blood is slightly elevated but not to the extent of the diabetes. pre - diabetes open an unpleasant door that may also lead to heart diseases or strokes. but here is a good news, you can prevent pre - diabetes from developing into type 2 disease


C:\Users\prati\AppData\Local\Temp/ipykernel_10844/3065070839.py:867: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  score.append(counter_q.similarity(nlp(ans1)))
C:\Users\prati\AppData\Local\Temp/ipykernel_10844/3065070839.py:868: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive t

In [3]:
ls= []
ls.append(q)
ls.append(p)
ls.append(r)
ls.append(s)
ls.append(t)

In [4]:
ls

[1,
 '   pre - diabetes, as the name suggests occurs before the diabetes phase and has the potential to evolve into a fully fledged type 2 diabetes down the road. the most frightening aspect of pre - diabetes is that it has no symptoms unless a manual blood test is done. during this condition, the body functions normally, just that the glucose level in the blood is slightly elevated but not to the extent of the diabetes. pre - diabetes open an unpleasant door that may also lead to heart diseases or strokes. but here is a good news, you can prevent pre - diabetes from developing into type 2 disease',
 '',
 '',
 '']